In [743]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from sklearn import *

In [744]:
class Preprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter
	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [745]:
class SimplePreprocessorGray:

	def preprocess(self, image):
		return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )

In [746]:
# Classe para carregar a base de imagens
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None):
    # store the image preprocessor
    self.preprocessors = preprocessors
		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []
		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      			# check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)
			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)

      # show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        print("[INFO] processed {}/{}".format(i + 1,
					len(imagePaths)))
		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

In [747]:
# Definindo os parametros
class Parametros:
  dataset = "./Imagens"
  neighbors = 1
  jobs = -1

args = vars(Parametros)

In [748]:
# grab the list of images that we'll be describing
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = Preprocessor(32, 32)
spg = SimplePreprocessorGray()
sdl = SimpleDatasetLoader(preprocessors=[spg])
(data, labels) = sdl.load(imagePaths, verbose=500)
#data = data.reshape((data.shape[0], 60200))

# show some information on memory consumption of the images
print("[INFO] features matrix: {:.1f}MB".format(
	data.nbytes / (1024 * 1024.0)))

[INFO] loading images...
[INFO] features matrix: 2.5MB


In [749]:
# Obtendo keypoints das imagens
sift = cv2.SIFT_create()
principais_caracteristicas = []
principais_labels = []
n_sample = 50 # 50 keypoints de cada img pra tbm nao ficar pesado

# For cada imagem
for idx, img in enumerate(data):
  # pega o sift de cada img
  keypoints, descriptors = sift.detectAndCompute(data[idx], None)

  principais = descriptors[np.random.randint(descriptors.shape[0], size=n_sample)]

  principais_caracteristicas[(idx*n_sample):(idx*n_sample)+n_sample] = principais[:]

In [785]:
# Construindo o vocabulario
print(len(data)) # total de imgs
print(len(principais)) # 50 keypoints por imagem
print(len(principais_caracteristicas)) # total de imagens * keypoints por imagem

vocab_size = len(principais_caracteristicas) // 20 # Vocabulario vai ter 10% das keypoints
vocab = KMeans(n_clusters=2).fit(principais_caracteristicas, vocab_size)

44
50
2200


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [786]:
# Obtendo histogramas da base de imgs X vocabulario das principais keypoints

histo_list = []

# For cada imagem
for idx, img in enumerate(data):
  # Obtenho novamente os keypoints da img
  keypoints, descriptors = sift.detectAndCompute(data[idx], None)

  # Verifico como o vocabulario enquadra eles, montando assim o histograma
  histo = np.zeros(n_sample)
  nkp = np.size(keypoints)

  for d in descriptors:
      idx = vocab.predict([d])
      histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

  histo_list.append(histo)

print("hist_list: ", len(histo_list))

hist_list:  44


In [787]:
# transforma as categorias em inteiros
le = LabelEncoder()
labels_number = le.fit_transform(labels)

trainX, testX, trainY, testY = train_test_split(histo_list, labels_number, test_size=0.3, random_state=42)

In [788]:
print(len(trainX))
print(len(trainY))

30
30


In [789]:
# Com esses histogramas, eu treino o meu modelo

knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(trainX, trainY)

KNeighborsClassifier(n_neighbors=3)

In [790]:
print(classification_report(testY, knn_model.predict(testX)))

              precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.71      0.62      0.67         8

    accuracy                           0.64        14
   macro avg       0.64      0.65      0.64        14
weighted avg       0.65      0.64      0.64        14



In [756]:
# Exportando
import pickle

pickle.dump(vocab, open("MontadorHistograma.pkl", "wb"))
pickle.dump(knn_model, open("LeitorHistograma.pkl", "wb"))